<div style="color: #095AAD; font-weight: bold; font-size: 16px;">

# Flight Delay & Cancellation Dashboard — Data Preparation  
</div>

This notebook contains the full data preparation process for a flight delay and cancellation dataset used in a dashboard project for the U.S. Department of Transportation.

The goal of this step is to ensure that the data is clean, consistent, and properly structured to support further analysis and interactive visualizations.

In this notebook, I will:

- Load and preview the flight dataset  
- Assign a unique Flight ID to each record for table joins in Tableau  
- Split multiple delay reasons per flight into separate rows  
- Create two additional dataframes for export and use in Tableau:
  1. A detailed table with `Flight ID`, `Delay Reason`, and `Delay Minutes`  
  2. A monthly summary table with `Target` (month name), `Source` (reason), `Value` (count), and `Month Number`

<div style="color: #095AAD; font-weight: bold; font-size: 16px;">

## Dataset Description  
</div>

The dataset contains operational flight information with disruption metrics for U.S. domestic flights in 2015. The data includes scheduled and actual performance, as well as causes of delays and cancellations.

| **Column Name**         | **Description**                                                                 |
|-------------------------|---------------------------------------------------------------------------------|
| `Airline`               | Carrier name or code                                                           |
| `Cancellation Reason`   | Reason for cancellation, coded as:  
&nbsp;&nbsp;&nbsp;&nbsp;- `A`: Airline (e.g. maintenance or crew problems)  
&nbsp;&nbsp;&nbsp;&nbsp;- `B`: Weather (e.g. storms, fog, visibility issues)  
&nbsp;&nbsp;&nbsp;&nbsp;- `C`: National Air System (e.g. air traffic control, capacity issues)  
&nbsp;&nbsp;&nbsp;&nbsp;- `D`: Security (e.g. threat, lockdowns, screening problems) |
| `DAY`                   | Day of the month (1–31)                                                        |
| `Day Of Week`           | Day of the week as a number (1 = Monday, ..., 7 = Sunday)                      |
| `Destination Airport`   | IATA code of the destination airport                                           |
| `Flight Number`         | Operating flight number                                                        |
| `Month`                 | Month of the year (1–12)                                                       |
| `Origin Airport`        | IATA code of the origin airport                                                |
| `Tail Number`           | Aircraft tail number                                                           |
| `Year`                  | Year of flight operation (always 2015)                                         |
| `Air System Delay`      | Delay caused by the national air system (minutes)                              |
| `Air Time`              | Time spent in the air (minutes)                                                |
| `Airline Delay`         | Delay attributed to the airline itself (minutes)                               |
| `Arrival Delay`         | Total delay in arrival time (minutes)                                          |
| `Arrival Time`          | Actual arrival time (HHMM)                                                     |
| `Cancelled`             | Binary flag: 1 = Cancelled, 0 = Not Cancelled                                  |
| `Departure Delay`       | Delay in scheduled departure (minutes)                                         |
| `Departure Time`        | Actual departure time (HHMM)                                                   |
| `Distance`              | Distance between origin and destination (miles)                                |
| `Diverted`              | Binary flag: 1 = Diverted, 0 = Not Diverted                                    |
| `Elapsed Time`          | Total elapsed time of flight (minutes)                                         |
| `Late Aircraft Delay`   | Delay due to late arrival of the same aircraft from a previous flight (minutes)|
| `Scheduled Arrival`     | Scheduled arrival time (HHMM)                                                  |
| `Scheduled Departure`   | Scheduled departure time (HHMM)                                                |
| `Scheduled Time`        | Scheduled flight duration (minutes)                                            |
| `Security Delay`        | Delay caused by security reasons (minutes)                                     |
| `Taxi In`               | Time spent taxiing to the gate after landing (minutes)                         |
| `Taxi Out`              | Time spent taxiing from the gate before takeoff (minutes)                      |
| `Weather Delay`         | Delay due to weather conditions (minutes)                                      |
| `Wheels Off`            | Time when aircraft wheels left the ground (HHMM)                               |
| `Wheels On`             | Time when aircraft wheels touched down (HHMM)                                  |

<div style="color: #095AAD; font-weight: bold; font-size: 15px;">

### Importing Required Libraries  
</div>

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

<div style="color: #095AAD; font-weight: bold; font-size: 15px;">

### Loading and Previewing the Dataset  
</div>

In [2]:
# Load the dataset
df = pd.read_csv(
    "data_for_dashboard/flights_light.csv",
    encoding="utf-16",
    sep="\t",
    dtype={"Cancellation Reason": str}
)

# Preview first rows
df.head()

,Airline,Cancellation Reason,DAY,Day Of Week,Destination Airport,Flight Number,Month,Origin Airport,Tail Number,Year,...,Late Aircraft Delay,Scheduled Arrival,Scheduled Departure,Scheduled Time,Security Delay,Taxi In,Taxi Out,Weather Delay,Wheels Off,Wheels On
0,US,NaN,19,1,CLT,624,1,SEA,N551UW,2015,...,NaN,603,2220,283,NaN,6.0,15.0,NaN,2233.0,543.0
1,US,NaN,19,1,RSW,1841,1,CLT,N556UW,2015,...,NaN,1620,1430,110,NaN,8.0,15.0,NaN,1437.0,1620.0
2,US,NaN,19,1,CLT,1772,1,LAS,N567UW,2015,...,NaN,802,59,243,NaN,8.0,17.0,NaN,109.0,738.0
3,US,NaN,19,1,LAX,1993,1,CLT,N190UW,2015,...,NaN,1030,755,335,NaN,8.0,19.0,NaN,816.0,1011.0
4,US,NaN,19,1,CLT,836,1,SJU,N292AY,2015,...,NaN,1819,1530,229,NaN,9.0,17.0,NaN,1544.0,1812.0


<div style="color: #095AAD; font-weight: bold; font-size: 15px;">

### Creating Flight ID Column  
</div>

Each flight will receive a unique identifier (`Flight ID`) that allows consistent joins between tables in Tableau.

In [3]:
# Create Flight ID and move it to the first column
df.insert(0, "Flight ID", range(1, len(df) + 1))

# Preview updated dataframe
df.head()

,Flight ID,Airline,Cancellation Reason,DAY,Day Of Week,Destination Airport,Flight Number,Month,Origin Airport,Tail Number,...,Late Aircraft Delay,Scheduled Arrival,Scheduled Departure,Scheduled Time,Security Delay,Taxi In,Taxi Out,Weather Delay,Wheels Off,Wheels On
0,1,US,NaN,19,1,CLT,624,1,SEA,N551UW,...,NaN,603,2220,283,NaN,6.0,15.0,NaN,2233.0,543.0
1,2,US,NaN,19,1,RSW,1841,1,CLT,N556UW,...,NaN,1620,1430,110,NaN,8.0,15.0,NaN,1437.0,1620.0
2,3,US,NaN,19,1,CLT,1772,1,LAS,N567UW,...,NaN,802,59,243,NaN,8.0,17.0,NaN,109.0,738.0
3,4,US,NaN,19,1,LAX,1993,1,CLT,N190UW,...,NaN,1030,755,335,NaN,8.0,19.0,NaN,816.0,1011.0
4,5,US,NaN,19,1,CLT,836,1,SJU,N292AY,...,NaN,1819,1530,229,NaN,9.0,17.0,NaN,1544.0,1812.0


<div style="color: #095AAD; font-weight: bold; font-size: 15px;">

### Finalizing the Working Dataset  
</div>

The dataset with the newly assigned `Flight ID` is saved as `with_id_flights_light.csv` in the `data_for_dashboard/` folder. This version will serve as the primary dataset for further transformations and Tableau visualizations.

In [4]:
# Save the dataset and preserve empty cells instead of filling NaNs with 0
df.to_csv(
    "data_for_dashboard/with_id_flights_light.csv",
    index=False,
    encoding="utf-8",
    na_rep="",
    float_format="%.0f"
)

<div style="color: #095AAD; font-weight: bold; font-size: 16px;">

## Create Delay Reason Table for Tableau</div>

To analyze delay causes per flight, we need to reshape the dataset.  
The original file contains separate columns for each reason with delay minutes.

In this step, I will:

- Unpivot five delay-related columns into rows using `melt()`
- Filter out rows with no delay (0 or NaN)
- Create a clean table with:
  - `Flight ID`
  - `Delay Reason`
  - `Delay Minutes`

In [5]:
# Define delay columns in alphabetical order
delay_cols_ordered = [
    "Air System Delay",
    "Airline Delay",
    "Late Aircraft Delay",
    "Security Delay",
    "Weather Delay"
]

# Unpivot delay columns into long format
df_delay = df[["Flight ID"] + delay_cols_ordered].melt(
    id_vars="Flight ID",
    var_name="Delay Reason",
    value_name="Delay Minutes"
)

# Keep only rows with non-zero delays
df_delay = df_delay[
    df_delay["Delay Minutes"].notna() & (df_delay["Delay Minutes"] > 0)
]

# Remove the word " Delay" from reason labels
df_delay.loc[:, "Delay Reason"] = df_delay["Delay Reason"].str.replace(" Delay", "", regex=False)

# Reset index
df_delay = df_delay.reset_index(drop=True)

# Preview the result
df_delay.head()

,Flight ID,Delay Reason,Delay Minutes
0,16,Air System,13.0
1,17,Air System,14.0
2,19,Air System,6.0
3,20,Air System,16.0
4,21,Air System,17.0


<div style="color: #095AAD; font-weight: bold; font-size: 16px;">

## Saving delay breakdown table</div>

I am saving the cleaned delay breakdown table with individual reasons per flight. This file will be used in Tableau for root cause visualizations.

In [6]:
# Save the dataset and preserve empty cells instead of filling NaNs with 0

df_delay.to_csv(
    "data_for_dashboard/with_id_delay_reasons.csv",
    index=False,
    encoding="utf-8",
    float_format="%.0f",
    na_rep=""
)

<div style="color: #095AAD; font-weight: bold; font-size: 16px;">

## Monthly Delay Reasons Summary Table</div>

This summary table is prepared specifically for building the **Sankey Flow Diagram** using an external tool. The output from this tool will later be imported into Tableau as a separate data source.

It allows comparing the number of delays by reason across months — either for the entire year or filtered by a specific month.

Each row represents a combination of **delay reason** (`Source`) and **month** (`Target`) with the total number of cases (`Value`). The `Month Number` column is included for proper sorting, since some tools require numerical ordering for time series.

In [7]:
# Extract only necessary columns from the original dataset
month_info = df[["Flight ID", "Month"]].copy()
month_info = month_info.rename(columns={"Month": "Month Number"})

# Merge with df_delay using "Flight ID"
df_delay_merged = df_delay.merge(month_info, on="Flight ID", how="left")

# Map month numbers to abbreviated names
import calendar
month_map = {i: calendar.month_abbr[i] for i in range(1, 13)}
df_delay_merged["Target"] = df_delay_merged["Month Number"].map(month_map)

# Group and aggregate
monthly_summary = df_delay_merged.groupby(
    ["Target", "Delay Reason", "Month Number"], as_index=False
).agg(Value=("Delay Reason", "count"))

# Rename and reorder columns
monthly_summary = monthly_summary.rename(columns={"Delay Reason": "Source"})
monthly_summary = monthly_summary[["Target", "Source", "Value", "Month Number"]]

# Sort by month number
monthly_summary = monthly_summary.sort_values(by="Month Number")

In [8]:
monthly_summary.head()

,Target,Source,Value,Month Number
24,Jan,Weather,285,1
23,Jan,Security,13,1
22,Jan,Late Aircraft,2177,1
21,Jan,Airline,2204,1
20,Jan,Air System,2403,1


<div style="color: #095AAD; font-weight: bold; font-size: 16px;">

## Save Monthly Delay Reasons Summary Table</div>

This table summarizes the total number of delay reasons by month.  
It is sorted by month number and structured specifically for building the Sankey Flow Diagram in Tableau.

We will save the final dataframe as `airflow_final_with_month_number.csv` for later import.

In [9]:
# Save the dataset and preserve empty cells instead of filling NaNs with 0

monthly_summary.to_csv(
    "data_for_dashboard/airflow_final_with_month_number.csv",
    index=False,
    encoding="utf-8",
    float_format='%.0f',
    na_rep=''
)

<div style="color: #095AAD; font-weight: bold; font-size: 16px;">

## Final Notes on Data Preparation</div>

In this notebook, I transformed the raw flight dataset to prepare it for advanced visualizations and insights in Tableau.  
No cleaning or validation was required, as the original data was already preprocessed. The focus was on reshaping the data for usability and compatibility.

**Key preparation steps included:**

- Assigning a unique `Flight ID` to each record for consistent table joins.
- Unpivoting multiple delay reason columns into a long format to enable flexible visualizations and drilldowns.
- Creating a detailed breakdown of delay reasons per flight.
- Generating a monthly summary table for building an external Sankey Flow Diagram, complete with both numeric and textual month formats for sorting and filtering flexibility.
- Ensuring that all exported datasets preserve original naming conventions and formatting required for Tableau integration.

These transformations make the dataset modular, scalable, and well-suited for both operational dashboards and exploratory data analysis.